In [1]:
cd ..

/Users/AchilleTalon/Documents/GitHub/picarro_calculator/picarro-calculator


In [2]:
import Picarro as pica
import PicarroDatabase as pdb

In [3]:
files = pdb.listFiles("example_data")

In [4]:
files

['example_data/HKDS2009_IsoWater_20191119_101154.csv',
 'example_data/HKDS2009_IsoWater_20191005_095530.csv',
 'example_data/HKDS2009_IsoWater_20191103_093239.csv',
 'example_data/HKDS2009_IsoWater_20191004_103528.csv',
 'example_data/HKDS2009_IsoWater_20191015_094044.csv',
 'example_data/HKDS2009_IsoWater_20191102_111519.csv',
 'example_data/HKDS2009_IsoWater_20191114_105042.csv',
 'example_data/HKDS2009_IsoWater_20191106_083502.csv',
 'example_data/HKDS2009_IsoWater_20191117_122133.csv',
 'example_data/HKDS2009_IsoWater_20191018_094838.csv',
 'example_data/HKDS2009_IsoWater_20191118_103014.csv',
 'example_data/HKDS2009_IsoWater_20191009_110451.csv',
 'example_data/HKDS2009_IsoWater_20191116_120411.csv',
 'example_data/HKDS2009_IsoWater_20191104_084429.csv',
 'example_data/HKDS2009_IsoWater_20191010_094459.csv',
 'example_data/HKDS2009_IsoWater_20191014_075543.csv',
 'example_data/HKDS2009_IsoWater_20191031_101623.csv']

In [5]:
conn = pdb.CreateConnection("database/isotopes.db")

for file in files:
    #pdb.AddSummaryRun(file,conn)
    pdb.AddRaw(file,conn)

table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
UNIQUE constraint failed: rawrun.Timestamp Mean
UNIQUE constraint failed: rawrun.Timestamp Mean
UNIQUE constraint failed: rawrun.Timestamp Mean
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists
table rawrun already exists


In [6]:
#pdb.AddRun("example_data/HKDS2009_IsoWater_20191005_095530.csv",conn)

In [7]:
import pandas as pd

In [10]:
df = pd.read_sql_query("select * from rawrun where RUN_ID = 20191005;",conn)

In [11]:
df

,Line,Analysis,Time Code,Port,Inj Nr,d(18_16)Mean,d(D_H)Mean,H2O_Mean,Ignore,Good,...,h16od_adjust,h16od_shift,n2_flag,DAS Temp,Tray,Sample,Job,Method,Error Code,RUN_ID
0,1,P-10229,2019/10/05 12:47:36,1-01,1,-17.366,-94.222,23638.0,-1,0,...,0.0,0.0,0,41.750,1,1,1,Picarro pre rinse,0,20191005
1,2,P-10229,2019/10/05 12:56:44,1-01,2,-16.182,-91.327,19536.0,-1,1,...,0.0,0.0,0,41.312,1,1,1,Picarro pre rinse,0,20191005
2,3,P-10229,2019/10/05 13:05:51,1-01,3,-16.490,-91.903,20130.0,-1,1,...,0.0,0.0,0,41.188,1,1,1,Picarro pre rinse,0,20191005
3,4,P-10229,2019/10/05 13:15:01,1-01,4,-16.358,-91.167,21294.0,0,1,...,0.0,0.0,0,41.125,1,1,1,Picarro pre rinse,0,20191005
4,5,P-10229,2019/10/05 13:24:09,1-01,5,-16.351,-91.446,22505.0,0,1,...,0.0,0.0,0,41.000,1,1,1,Picarro pre rinse,0,20191005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,128,P-10254,2019/10/06 08:07:28,1-26,4,-15.459,-85.066,20376.0,0,1,...,0.0,0.0,0,40.625,1,26,2,Picarro pre rinse,0,20191005
128,129,P-10255,2019/10/06 08:16:34,1-27,1,-16.190,-89.906,20193.0,-1,1,...,0.0,0.0,0,40.688,1,27,2,Picarro pre rinse,0,20191005
129,130,P-10255,2019/10/06 08:25:43,1-27,2,-16.388,-91.663,20075.0,-1,1,...,0.0,0.0,0,40.688,1,27,2,Picarro pre rinse,0,20191005
130,131,P-10255,2019/10/06 08:34:50,1-27,3,-16.377,-91.881,19754.0,-1,1,...,0.0,0.0,0,40.688,1,27,2,Picarro pre rinse,0,20191005
